In [1]:
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import h5py

In [2]:
with h5py.File('X_train.h5', 'r') as hf:
    X = hf['X_train'][:]

with h5py.File('Y_train.h5', 'r') as hf:
    Y = hf['Y_train'][:]

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=87)

In [4]:
import tensorflow as tf

In [5]:
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Attention
from tensorflow.keras.datasets import imdb

In [7]:
X.shape

(77526, 15, 28)

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=87)


In [4]:
(x_train,y_train), (x_test, y_test) = imdb.load_data(num_words=15000)

In [5]:
x_train = sequence.pad_sequences(x_train, maxlen = 150)
x_test = sequence.pad_sequences(x_test, maxlen = 150)

In [9]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [53]:
main_input = Input(shape=( 15,28 ), name='main_input')

In [10]:
x1 = Embedding(output_dim=512, input_dim=15000, input_length=150)(main_input)
x2 = Embedding(output_dim=512, input_dim=15000, input_length=150)(main_input)

In [54]:
query_value_attention_seq = tf.keras.layers.Attention()(
    [main_input, main_input])

# Reduce over the sequence axis to produce encodings of shape
# [batch_size, filters].
query_encoding = tf.keras.layers.GlobalAveragePooling1D()(
    main_input)
query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(
    query_value_attention_seq)

#Concatenate query and document encodings to produce a DNN input layer.
input_layer = tf.keras.layers.Concatenate()(
    [query_encoding, query_value_attention])

In [55]:
x = Dense(64, activation='relu')(input_layer)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
#得到输出的张量
# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [58]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 15, 28)]     0                                            
__________________________________________________________________________________________________
attention_10 (Attention)        (None, 15, 28)       0           main_input[0][0]                 
                                                                 main_input[0][0]                 
__________________________________________________________________________________________________
global_average_pooling1d_7 (Glo (None, 28)           0           main_input[0][0]                 
__________________________________________________________________________________________________
global_average_pooling1d_8 (Glo (None, 28)           0           attention_10[0][0]         

In [57]:
model = Model(inputs=main_input, outputs=main_output)
model.compile(optimizer='rmsprop', loss='binary_crossentropy')

In [59]:
Y.shape

(77526, 1)

In [60]:
model.fit(X_train,Y_train ,epochs=3, batch_size=32)

Train on 54268 samples
Epoch 1/3
54268/54268 [==============================] - 12s 226us/sample - loss: nan
Epoch 2/3
54268/54268 [==============================] - 10s 185us/sample - loss: nan
Epoch 3/3
54268/54268 [==============================] - 6s 104us/sample - loss: nan


In [21]:
model.evaluate(x_test,y_test)

25000/25000 [==============================] - 62s 2ms/sample - loss: 0.3193


0.31927722802162173